In [90]:
# llama tool set 
from llama_index import LlamaIndex
from llama_index.core import SQLDatabase
import json
import openai
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    column,
    Text,
)

ImportError: cannot import name 'LlamaIndex' from 'llama_index' (unknown location)

In [79]:
# sets data for our json file
with open('data/EHR-SeqSQL-data.json','r') as file:
    data = json.load(file)
#print(data[:2])
#print(data[:1][0].keys())# visualizes the data
#print(data[0]['value'])

In [80]:
engine = create_engine("sqlite:///:EHR-SEQSQL.db:", future=True)
#creating the sql tables
metadata_obj = MetaData()
patient_stats_table = Table(
    "patient_stats",
    metadata_obj,
    Column("id", String(36), primary_key=True),
    Column("seed_question", Text, nullable=False),
    Column("value", Text, nullable=False),
    Column("question", Text, nullable=False),
    Column("question_template", Text, nullable=False),
    Column("sql", Text, nullable=False),
    Column("seqsql", Text, nullable=False),
    Column("department", Text, nullable=False),
    Column("importance", String(10), nullable=False),
    Column("compositional_split", String(10), nullable=False),
    Column("random_split", String(10), nullable=False),
)

In [81]:
# Create the table in the database
metadata_obj.create_all(engine)

In [82]:
#inputting the data 
with engine.connect() as connection:
    for record in data:
        # Flatten the dictionary for insertion
        #print(record)
            value = next(iter(record['value'].values()), '') 
            flattened_record = {
                'id': record['id'],
                'seed_question': record['seed_question'],
                #'value': record['value']['drug_name'],  # Extract drug name
                'value': value,
                'question': record['question'][0],       # Take the first question
                'question_template': record['question_template'][0],  # First template
                'sql': record['sql'][0],                 # First SQL query
                'seqsql': record['seqsql'][0],           # First seqsql
                'department': record['department'],
                'importance': record['importance'],
                'compositional_split': record['compositional_split'],
                'random_split': record['random_split'],
            }
            connection.execute(patient_stats_table.insert().values(flattened_record).prefix_with("OR REPLACE"))



In [83]:
print(flattened_record) 

{'id': 'be6a88096e737997f72e2690', 'seed_question': 'give me the top five most frequently tested cultures that patients took during the same hospital visit after being diagnosed with fx mult ribs nos-closed since 2105.', 'value': 'fx mult ribs nos-closed', 'question': 'List all hospitalization ids, patient ids, and diagnosis times for fx mult ribs nos-closed cases since 2105.', 'question_template': 'List all patient ids, time of diagnoses and hospitalization ids associated with diagnosis named fx mult ribs nos-closed since 2105.', 'sql': "SELECT admissions.subject_id, diagnoses_icd.charttime, admissions.hadm_id FROM diagnoses_icd JOIN admissions ON diagnoses_icd.hadm_id = admissions.hadm_id WHERE diagnoses_icd.icd9_code = ( SELECT d_icd_diagnoses.icd9_code FROM d_icd_diagnoses WHERE d_icd_diagnoses.short_title = 'fx mult ribs nos-closed' ) AND strftime('%Y',diagnoses_icd.charttime) >= '2105'", 'seqsql': "SELECT admissions.subject_id, diagnoses_icd.charttime, admissions.hadm_id FROM dia

In [84]:
with engine.connect() as connection:
    result = connection.execute(select(patient_stats_table)).fetchall()
    print(f"Total records in table: {len(result)}")



Total records in table: 0


In [85]:
# Text to Sql
def translate_text_to_sql(query):
    relevant_docs = index.query(query)
    sql_queries = [doc['sql'] for doc in relevant_docs if 'sql' in doc]
    return sql_queries

In [86]:
# Execute the SQL Queries
def execute_query(sql_query):
    with engine.connect() as connection:
        result = connection.execute(sql_query).fetchall()
        return result

In [89]:
index = LlamaIndex()
user_query = "What is the intake method of send 500mg vial?"
sql_queries = translate_text_to_sql(user_query)

if sql_queries:
    for sql_query in sql_queries:
        results = execute_query(sql_query)
        print("Results for query:", sql_query)
        for row in results:
            print(row)
else:
    print("No valid SQL query generated.")

NameError: name 'LlamaIndex' is not defined

In [91]:
# llm integration
def generate_sql_from_llm(user_query):
    prompt = f"Translate the following question into an SQL query: {user_query}"
    
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # or any other model you're using
        messages=[{"role": "user", "content": prompt}]
    )
    
    sql_query = response['choices'][0]['message']['content']
    return sql_query


In [92]:
def translate_text_to_sql(user_query):
    # Use LLM to generate SQL
    sql_query = generate_sql_from_llm(user_query)
    return sql_query
